In [1]:
testcase_list = ["Religion Unity", "Race Polarization", "Race Artistry", "Race Bilingual", "Religion Healing",
                 "Race Identity", "Royal Betrayal", "Religion Prayer", "Religion Values", "Royal Heritage"]

In [9]:
import os
from eventregistry import EventRegistry, QueryArticlesIter

import sys
sys.path.append('../src')
print(sys.path)
from LLM import topic_expander, filtering

api_key: str =  os.getenv("NEWS_API_KEY", "")

def fetch_articles(input_dict):

    # Initialize the EventRegistry client
    er = EventRegistry(apiKey=api_key)

    # from_date = input_dict["from_date"]
    # to_date = input_dict["to_date"]
    # max_news = input_dict["max_news"]

    articles_list = []

   # Extract and join category, sub_category, and topic
    topic_dict = {key: input_dict[key] for key in ["sub_category", "topic"]}
    topic_to_expand = "".join(value for value in topic_dict.values() if value)

    # expands the topic
    news_query = topic_expander(topic_to_expand)
    print ("This is the news query =", news_query)
    
    # Define the complex query with the topic
    query = {
        "$query": {
            "$and": [
            {
                "keyword": news_query,
                "keywordSearchMode": "simple"
            },
            {
                # "dateStart": from_date,
                # "dateEnd": to_date,
                "lang": "eng"
            }
            ]
        },
        "$filter": {
            "forceMaxDataTimeWindow": "31",
            "isDuplicate": "skipDuplicates"
        }
        }
    
    # Initialize the query with the complex query structure
    q = QueryArticlesIter.initWithComplexQuery(query)

    # Fetch articles and add them to the list
    for article in q.execQuery(er, maxItems=10):
        articles_list.append({
            "topic": topic_dict["topic"],
            "expanded_query": news_query,
            "title": article.get("title"),
            "url": article.get("url"),
            "published_date": article.get("dateTime"),
            "source": article.get("source", {}).get("title"),
            "content": article.get("body", "Full content not available")
        })

        # print("article = ", article)

    # PostProcessing - Filtering Relevant
    filtered_article_list = filtering(articles_list, news_query)

    for article in filtered_article_list:
        print ("filtered article = ", article)
        
    return filtered_article_list



['/Users/ayeshahussain/anaconda3/lib/python311.zip', '/Users/ayeshahussain/anaconda3/lib/python3.11', '/Users/ayeshahussain/anaconda3/lib/python3.11/lib-dynload', '', '/Users/ayeshahussain/Documents/Projects/00_MAIN_AGENTS/06 RADAR/newsapi/venv/lib/python3.11/site-packages', '../src', '../src', '../src', '../src', '../src', '../src', '../src', '../src']


In [10]:
testcase_article_list = []

for topic in testcase_list:
    input_dict = {'sub_category': 'hate speech', 'topic': topic}
    result = fetch_articles(input_dict)
    testcase_article_list.append(result)

print (testcase_article_list)

The topic to expand =  hate speechReligion Unity
['hate speech and religion unity', 'religious hate speech and unity efforts', 'unity in diversity and combating hate speech', 'hate speech against religious minorities and unity initiatives', 'promoting religion unity and reducing hate speech online']
This is the news query = hate speech and religion unity
unfiltered news article titles =  Dharm Sansad: Hindu Religious leaders condemn 'divisive' event even as a contempt petition is filed in SC | SabrangIndia
unfiltered news article titles =  Christian Alarm Over State Hate Speech Bill
unfiltered news article titles =  Constitution Day 2024: History, Importance, Speech Ideas and Famous Quotes
unfiltered news article titles =  You Deserve To Be Safe In Our State - Free Of Hate
unfiltered news article titles =  'Winter Whacking': PM Modi lambasts Gandhi family; lists govt's achievements
unfiltered news article titles =  Why do at least six anti-LGBTQIA+ hate groups have special consultative

In [11]:
import json
# Save to a JSON file
with open('articles.json', 'w') as json_file:
    json.dump(testcase_article_list, json_file, indent=4)